<a href="https://colab.research.google.com/github/kopaprin/shogi_viz/blob/master/shogi_profile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/shogi_pj/01_profile")
os.getcwd() 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


'/content/drive/My Drive/shogi_pj/01_profile'

In [0]:
#!pip install requests

import sys
import requests
from bs4 import BeautifulSoup
import re
import urllib.parse
import urllib.robotparser

import numpy as np
import pandas as pd
from tqdm import tqdm



import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
import time



def AngleBrackets(str_): # 山かっこ内だけ正規表現で抽出する
  result = re.search(r'>(.*?)<',str_).group(0)[1:][:-1]
  return result



In [0]:

url = "https://www.shogi.or.jp/player/"
r = requests.get(url)
r.encoding = r.apparent_encoding # r.encodingを上書きする
print(r.headers)
print(f'encoding:{r.encoding}')
print("---top 100 character-----------------------------------")
print(r.text[:100])

# shogi_playerとしてHTMLを保存
with open('shogi_player.html', 'w') as file:
    file.write(r.text)

{'Content-Encoding': 'gzip', 'Content-Type': 'text/html', 'Date': 'Wed, 12 Feb 2020 14:34:22 GMT', 'Server': 'nginx', 'Strict-Transport-Security': 'max-age=15768000', 'Content-Length': '11335', 'Connection': 'keep-alive'}
encoding:UTF-8-SIG
---top 100 character-----------------------------------
<!DOCTYPE HTML>
<html lang="ja"><!-- InstanceBegin template="/Templates/base.dwt" codeOutsideHTMLIsL


In [0]:
 soup = BeautifulSoup(r.text,"html.parser")

In [0]:
# 基本テーブルの作成OK

def MakeBaseTable():
  url = "https://www.shogi.or.jp/player/"
  r = requests.get(url)
  r.encoding = r.apparent_encoding # r.encodingを上書きする

  soup = BeautifulSoup(r.text,"html.parser")

  # Make img&player url
  img_list = []
  html_list = []
  player_list = []
  url_prefix = "https://www.shogi.or.jp/"
  for n in soup.select(".section04 > div > div"):
    # 01 Make img url
    img = re.search('images(.*?jpg)',str(n.select("figure.image > a > img")[0])).group(0)
    img_result = urllib.parse.urljoin(url_prefix,img)
    img_list.append(img_result)
    # 02 Make detail url
    html = re.search(r'player(.*?)html',str(n.select(".ttl > a[href]")[0])).group(0)
    html_result = urllib.parse.urljoin(url_prefix,html)
    html_list.append(html_result)
    # 03 get name
    player = re.search(r'>(.*?)<',str(n.select(".ttl > a")[0])).group(0)
    player_result = player[1:][:-1]
    player_list.append(player_result)

  profile_df = pd.DataFrame({
      "name": player_list,
      "profile": html_list,
      "img" :img_list
  })

  # 対局結果テーブルの取得
  table = pd.read_html("https://www.shogi.or.jp/game/record/year_result.html")[0]
  table.columns = (["name","match","win","lose","rate"])
  
  result = pd.merge(profile_df, table,on="name",how="left")

  return result
    
  
shogi_df = MakeBaseTable()



In [0]:
def MakeProfile(df):

  id_list = []
  name_list = []
  name_roma_list = []
  birth_day_list = []
  age_list = []
  teacher_list = []
  ranking_list = []

  for person_url in tqdm(df["profile"]):
    
    r = requests.get(person_url)
    r.encoding = r.apparent_encoding # r.encodingを上書きする

    soup = BeautifulSoup(r.text,"html.parser")

    id_ = soup.select("#contents > div:nth-of-type(2) > div > div.innerBlock01 > table > tbody > tr:nth-of-type(1) > td")
    id_list.append(AngleBrackets(str(id_)))
    name = soup.select("#contents > div:nth-of-type(2) > div > div.innerBlock01 > div > div > h1 > span.jp")
    name_list.append(AngleBrackets(str(name)))
    name_roma = soup.select("#contents > div:nth-of-type(2) > div > div.innerBlock01 > div > div > h1 > span.en")
    name_roma_list.append(AngleBrackets(str(name_roma)))
    birth_day = soup.select("#contents > div:nth-of-type(2) > div > div.innerBlock01 > table > tbody > tr:nth-of-type(2) > td")
    birth_day = re.search(r'>(.*?)\(',str(birth_day)).group(0)[1:][:-1]
    birth_day = datetime.datetime.strptime(birth_day, '%Y年%m月%d日')
    birth_day_list.append(birth_day)
    age_list.append(relativedelta( date.today(),birth_day).years) # 年齢計算プログラム
    teacher = soup.select("#contents > div:nth-of-type(2) > div > div.innerBlock01 > table > tbody > tr:nth-of-type(4) > td")
    teacher_list.append(AngleBrackets(str(teacher)))
    ranking = soup.select("#contents > div:nth-of-type(2) > div > div.innerBlock01 > table > tbody > tr:nth-of-type(6) > td > a")
    ranking_list.append(AngleBrackets(str(ranking)))

    time.sleep(1)
  df["id"] = id_list
  df["name_roma"] = name_roma_list
  df["birth_day"] = birth_day_list
  df["age"] = age_list
  df["teacher"] = teacher_list
  df["ranking"] = ranking_list

  return df

shogi_df = MakeProfile(shogi_df)

100%|██████████| 168/168 [05:30<00:00,  1.81s/it]


In [0]:
shogi_df.head()

,name,profile,img,match,win,lose,rate,id,name_roma,birth_day,age,teacher,ranking
0,豊島将之,https://www.shogi.or.jp/player/pro/264.html,https://www.shogi.or.jp/images/player/pro/264.jpg,51.0,34.0,17.0,0.6666,264,Masayuki Toyoshima,1990-04-30,29,桐山清澄九段,第77期名人（A級以上：3期）
1,渡辺明,https://www.shogi.or.jp/player/pro/235.html,https://www.shogi.or.jp/images/player/pro/235.jpg,45.0,36.0,9.0,0.8000,235,Akira Watanabe,1984-04-23,35,所司和晴七段,A級（A級：9期）
2,永瀬拓矢,https://www.shogi.or.jp/player/pro/276.html,https://www.shogi.or.jp/images/player/pro/276.jpg,43.0,32.0,11.0,0.7441,276,Takuya Nagase,1992-09-05,27,安恵照剛八段,B級1組
3,木村一基,https://www.shogi.or.jp/player/pro/222.html,https://www.shogi.or.jp/images/player/pro/222.jpg,38.0,20.0,18.0,0.5263,222,Kazuki Kimura,1973-06-23,46,（故）佐瀬勇次名誉九段,A級（A級：5期）
4,羽生善治,https://www.shogi.or.jp/player/pro/175.html,https://www.shogi.or.jp/images/player/pro/175.jpg,44.0,26.0,18.0,0.5909,175,Yoshiharu Habu,1970-09-27,49,(故)二上達也九段,A級（A級以上：27期)


In [0]:
shogi_df.to_csv("shogi_profile.csv",encoding="utf-8",header=True,index=False)

In [0]:
d_dir = "/content/drive/My Drive/shogi_pj/03_img"
sleep_time_sec = 1

def download_file(url, dst_path):
    try:
        with urllib.request.urlopen(url) as web_file, open(dst_path, 'wb') as local_file:
            local_file.write(web_file.read())
    except urllib.error.URLError as e:
        print(e)

def download_file_to_dir(url, dst_dir):
    download_file(url, os.path.join(dst_dir, os.path.basename(url)))

for url in shogi_df["img"]:
    download_file_to_dir(url, d_dir)
    time.sleep(sleep_time_sec)